In [1]:
import pandas as pd
import numpy as np
import scipy

Using Theano backend.


In [2]:
import theano

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy
from scipy import ndimage
import glob

In [5]:
from skimage.io import imread
# im = imread('../../DSI-SF-4-Rashimk/Cap_Stone_Project/test_mine/img_00007.jpg')

In [12]:
# Extracting and converting the images from test_mine into sobel(black and white format) which saves it to untitled
# folder in the desktop
      
path = r'../Cap_Stone_Project/test_mine/'
filenames = glob.glob(path + "/*.jpg")

num = 0
for filename in filenames:
    print filename
    im = scipy.misc.imread(filename)
    im = im.astype('int32')
    dx = ndimage.sobel(im, 1)  # horizontal derivative
    dy = ndimage.sobel(im, 0)  # vertical derivative
    mag = numpy.hypot(dx, dy)  # magnitude
    mag *= 240.0 / numpy.max(mag)
    scipy.misc.imsave('../../untitled folder/'+str(num)+'.jpg', mag)
    num = num + 1
    

../Cap_Stone_Project/test_mine/img_00005.jpg
../Cap_Stone_Project/test_mine/img_00007.jpg
../Cap_Stone_Project/test_mine/img_00009.jpg
../Cap_Stone_Project/test_mine/img_00018.jpg
../Cap_Stone_Project/test_mine/img_00027.jpg
../Cap_Stone_Project/test_mine/img_00030.jpg
../Cap_Stone_Project/test_mine/img_00040.jpg
../Cap_Stone_Project/test_mine/img_00046.jpg
../Cap_Stone_Project/test_mine/img_00053.jpg
../Cap_Stone_Project/test_mine/img_00071.jpg
../Cap_Stone_Project/test_mine/img_00075.jpg
../Cap_Stone_Project/test_mine/img_00102.jpg
../Cap_Stone_Project/test_mine/img_00103.jpg
../Cap_Stone_Project/test_mine/img_00109.jpg
../Cap_Stone_Project/test_mine/img_00119.jpg
../Cap_Stone_Project/test_mine/img_00120.jpg
../Cap_Stone_Project/test_mine/img_00125.jpg
../Cap_Stone_Project/test_mine/img_00128.jpg
../Cap_Stone_Project/test_mine/img_00129.jpg
../Cap_Stone_Project/test_mine/img_00133.jpg
../Cap_Stone_Project/test_mine/img_00138.jpg
../Cap_Stone_Project/test_mine/img_00141.jpg
../Cap_Sto

In [36]:
test_image = []
train_image = []

path = r'../Cap_Stone_Project/test_mine/'
filenames = glob.glob(path + "/*.jpg")

path1 = r'../../../Desktop/fish_train/'
filenames1 = glob.glob(path1 + "/*.jpg")


for filename in filenames:
    test_image.append(mpimg.imread(filename))
    
for filename in filenames1:
    train_image.append(mpimg.imread(filename))

In [37]:
# try to figure these out later
# from skimage.data import camera
# from skimage.filters import roberts, sobel, scharr, prewitt


In [38]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


In [40]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

In [41]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [42]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


In [43]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [44]:
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        '../../DSI-SF-4-Rashimk/Cap_Stone_Project/DATA/Train/',                                       # this is the target directory
        target_size=(150, 150),                             # all images will be resized to 150x150
        batch_size=32,
        class_mode='binary')                                # since we use binary_crossentropy loss, 
                                                            # we need binary labels


Found 3665 images belonging to 8 classes.


In [45]:
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '../../../Desktop/DSI-SF-4-Rashimk/Cap_Stone_Project/DATA/Validation1/',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

Found 112 images belonging to 8 classes.


In [46]:
model.fit_generator(
        train_generator,
        samples_per_epoch=2000,
        nb_epoch=5,
        validation_data=validation_generator,
        nb_val_samples=800)
model.save_weights('../Cap_Stone_Project/first_try1.h5')  # always save your weights after training or during training

Epoch 1/5
2016/2000 [==============================] - 205s - loss: -25.7240 - acc: 0.0551 - val_loss: -40.6765 - val_acc: 0.1348
Epoch 2/5
2001/2000 [==============================] - 178s - loss: -26.8336 - acc: 0.0485 - val_loss: -40.6765 - val_acc: 0.1348
Epoch 3/5
2016/2000 [==============================] - 179s - loss: -26.6814 - acc: 0.0486 - val_loss: -41.0087 - val_acc: 0.1360
Epoch 4/5
2001/2000 [==============================] - 182s - loss: -26.1404 - acc: 0.0510 - val_loss: -40.5984 - val_acc: 0.1311
Epoch 5/5
2016/2000 [==============================] - 186s - loss: -27.3931 - acc: 0.0496 - val_loss: -40.6570 - val_acc: 0.1336
